In [1]:
import pickle
import pandas as pd
import numpy as np
import torch
train_data = pickle.load(open('embeddings', 'rb'))

/backup/asen/vitaly/ml/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import gc
gc.collect()

0

In [27]:
emoji_stats = [
    ('👍', 14189),
    ('\U0001fae1', 76),
    ('🙏', 157),
    ('🔥', 5659),
    ('🥰', 1186),
    ('👌', 69),
    ('🤓', 58),
    ('🤝', 3),
    ('👎', 2049),
    ('🤬', 182),
    ('😁', 2405),
    ('🤡', 1453),
    ('🏆', 290),
    ('👏', 320),
    ('💯', 160),
    ('❤', 4263),
    ('🌚', 254),
    ('🤨', 293),
    ('🥴', 274),
    ('🤩', 238),
    ('😍', 64),
    ('🤣', 348),
    ('😢', 2800),
    ('💩', 1169),
    ('🤯', 598),
    ('❤\u200d🔥', 596),
    ('🐳', 457),
    ('🤮', 1203),
    ('🤗', 2),
    ('😇', 20),
    ('🤔', 964),
    ('🖕', 114),
    ('🥱', 93),
    ('😈', 213),
    ('🕊', 118),
    ('🍌', 476),
    ('🌭', 234),
    ('💋', 219),
    ('⚡', 83),
    ('🍓', 113),
    ('🍾', 287),
    ('💔', 38),
    ('😱', 442),
    ('🎉', 731),
    ('😐', 76),
    ('✍', 34),
    ('😭', 116),
    ('🆒', 31),
    ('🗿', 7),
    ('👀', 48),
    ('💅', 6),
    ('🎄', 66),
    ('☃', 3),
    ('👨\u200d💻', 2),
    ('👻', 7),
    ('🙊', 1),
    ('🤪', 4),
    ('😨', 2),
    ('💊', 2),
    ('😴', 2),
]
class_weights = {}
for emoji_id in range(60):
    class_weights[emoji_id] = emoji_stats[emoji_id][1]

In [28]:
#!pip install xgboost
import lightgbm
import xgboost
n_estimators = 25
learning_rate = 0.03
model = lightgbm.LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, class_weight=class_weights)

In [29]:
target_data = pd.read_csv('data_all.csv')

In [30]:
target_data = target_data.drop(['text', 'total', 'Unnamed: 52'], axis=1)
target_data_np = np.array(target_data)
target = np.argmax(target_data_np, axis=1)
target_emojis = pd.Series(list(map(lambda x:target_data.columns[x], target)))
target_data.head(), train_data.head()

(     👍    🫡    🙏    🔥    🥰    👌    🤓    🤝    👎    🤬  ...    💅    🎄    ☃  👨‍💻  \
 0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
 1  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
 2  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
 3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
 4  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
 
      👻    🙊    🤪    😨    💊    😴  
 0  0.0  0.0  0.0  0.0  0.0  0.0  
 1  0.0  0.0  0.0  0.0  0.0  0.0  
 2  0.0  0.0  0.0  0.0  0.0  0.0  
 3  0.0  0.0  0.0  0.0  0.0  0.0  
 4  0.0  0.0  0.0  0.0  0.0  0.0  
 
 [5 rows x 60 columns],
       0         1         2         3         4         5         6      \
 0  0.108233  0.020312 -0.126104  0.031688  0.194239  0.108156 -0.056828   
 1  0.020765 -0.049883 -0.030293  0.036535  0.296358  0.146200  0.008478   
 2  0.154851  0.069561  0.103274  0.183172  0.16

In [41]:
from sklearn.model_selection import train_test_split
#train_data.drop(['text'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.05)
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
X_train.shape, X_test.shape, len(y_train), len(y_test), target[:5]

((7969, 98305), (420, 98305), 7969, 420, array([0, 1, 2, 0, 0]))

In [49]:
bad_ids = []
for emoji_index in keys:
    if sum(y_train == emoji_index) == 0:
        bad_ids.append(emoji_index)
for emoji_id in bad_ids:
    del class_weights[emoji_id]

In [50]:
from sklearn.metrics import accuracy_score

In [51]:
model.fit(X_train, y_train)

LGBMClassifier(class_weight={0: 14189, 1: 76, 2: 157, 3: 5659, 4: 1186, 5: 69,
                             6: 58, 7: 3, 8: 2049, 9: 182, 10: 2405, 11: 1453,
                             12: 290, 13: 320, 14: 160, 15: 4263, 16: 254,
                             17: 293, 18: 274, 19: 238, 20: 64, 21: 348,
                             22: 2800, 23: 1169, 24: 598, 25: 596, 26: 457,
                             27: 1203, 28: 2, 29: 20, ...},
               learning_rate=0.03, n_estimators=25)

In [52]:
preds = model.predict(X_test)
print(accuracy_score(y_test, preds))

0.4880952380952381


In [53]:
f_model = open('LGBM_model', 'wb')
pickle.dump(model, f_model)
f_model.close()

In [54]:
from sklearn.metrics import recall_score
y_train_default = (y_train == 0)
model_default = lightgbm.LGBMClassifier(class_weight='balanced')
model_default.fit(X_train, y_train_default)
preds = model.predict(X_test)
preds_default = model_default.predict(X_test)
preds_result = preds == 0
true_result = y_test == 0
print(accuracy_score(preds, y_test), recall_score(preds_result, true_result))

0.4880952380952381 0.5088161209068011


In [55]:
f_model_default = open('LGBM_model_default', 'wb')
pickle.dump(model_default, f_model_default)
f_model_default.close()